# Time Series Forecasting usinhg Transformer: Single-time-step Predictions


In this notebook, we describe the steps for single-time-step predictions for a wesather forecasting problem. We train a Transformer model for making predictions.

The notebook is adapted from the following two Keras tutorials.
- https://keras.io/examples/timeseries/timeseries_weather_forecasting/
- https://keras.io/examples/timeseries/timeseries_classification_transformer/


## Dataset
We use the the Jena Climate dataset by the Max Planck Institute for Biogeochemistry.
https://www.bgc-jena.mpg.de/wetter/

The dataset has 15 features such as datetime, temperature, pressure, humidity, etc. There are 420551 observations (time steps), each recorded once per 10 minutes (6 times per hour).

- Total observations = 420551 (an observation is recorded per 10 minutes)
- Total features = 15

Though 6 observations are recorded every hour, we use only one observation per hour to train the model since no drastic change is expected within an hour. 


## Single-time-step Predictions

For the single-time-step predictions, we need to decide the length of each sequence and the a future location on the sequence that we want to predict.

We chose the sequence length to be 120 hours (on the 10 minutes recording scale, the length spans 720 observations).

We want to predict an observation which is 12 hours (on the 10 minutes recording scale, it is 72 observations) future from the last observation in the sequence. This gap is added to make the prediction slightly non-trivial. 

The first training sequence is as follows (though we use only one observation per hour):

        0 1 2 ... 719

The index of the location of the first prediction (720 training observations + 72 "gap" observations):
        
        792



For training the model, we use ~300,000 samples (observations). These observations are divided into constant length sequences.
- Each sequence length = 120 hours (720 observations)
- Every sequence has a label (after the 72 observations)

Therefore,
- A sample = 120 hours (720 observations) + 1 label
- A batch contains multiple samples.


In [ ]:
from zipfile import ZipFile
import os

import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Load the Dataset

In [ ]:
'''
Download the Dataset
'''
uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = tf.keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"


'''
Load the dataset as a Pandas DataFRame object
'''
df = pd.read_csv(csv_path)
print("Dimension of the data: ", df.shape)
df.info()

## Raw Data Visualization

We plot each feature pattern over the time period from 2009 to 2016. The plots help to identify the anomalies.

In [ ]:
titles = [
    "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",
]

feature_keys = [
    "p (mbar)",
    "T (degC)",
    "Tpot (K)",
    "Tdew (degC)",
    "rh (%)",
    "VPmax (mbar)",
    "VPact (mbar)",
    "VPdef (mbar)",
    "sh (g/kg)",
    "H2OC (mmol/mol)",
    "rho (g/m**3)",
    "wv (m/s)",
    "max. wv (m/s)",
    "wd (deg)",
]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]

date_time_key = "Date Time"


def show_raw_visualization(data):
    time_data = data[date_time_key]
    fig, axes = plt.subplots(
        nrows=7, ncols=2, figsize=(15, 20), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(len(feature_keys)):
        key = feature_keys[i]
        c = colors[i % (len(colors))]
        t_data = data[key]
        t_data.index = time_data
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{} - {}".format(titles[i], key),
            rot=25,
        )
        ax.legend([titles[i]])
    plt.tight_layout()


show_raw_visualization(df)

## Feature Correlation

We create a heat map to show the correlation between different features.

In [ ]:
def show_heatmap(data):
    plt.matshow(data.corr())
    plt.xticks(range(data.shape[1]), data.columns, fontsize=14, rotation=90)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(range(data.shape[1]), data.columns, fontsize=14)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Feature Correlation Heatmap", fontsize=14)
    plt.show()


show_heatmap(df)

## Data Preprocessing

As mentioned earlier, we use one obervation per hour by resampling from 6 observations/hour.

Since the length of a sequence is 120, we use 720 observations-long sequence for prediction (training). The prediction location on the sequence is after 12 hours or 72 observations. This observation is used as a label.


### Normalization
Since every feature has values with varying ranges, we do normalization to confine feature values to a range of [0, 1] before training a neural network. We do this by subtracting the mean and dividing by the standard deviation of each feature.

### Train-Test Split
71.5 % of the data will be used to train the model, i.e. 300,693 rows. 

In [ ]:
split_fraction = 0.715
train_split = int(split_fraction * int(df.shape[0]))
step = 6  # It is required to determine the length of a sequence

past = 720  # Length of a sequence on the original observations (5 days)
future = 72  # Prediction is made after 72 observations (12 hours)

learning_rate = 0.001
batch_size = 256
epochs = 50


def normalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data - data_mean) / data_std

## Feature Selection

From the correlation heatmap, we identify that few parameters like Relative Humidity and Specific Humidity are redundant. This, we select only the relevant features for learning.

In [ ]:
print(
    "The selected parameters are:",
    ", ".join([titles[i] for i in [0, 1, 5, 7, 8, 10, 11]]),
)
selected_features = [feature_keys[i] for i in [0, 1, 5, 7, 8, 10, 11]]
features = df[selected_features]
features.index = df[date_time_key]
features.head()

features = normalize(features.values, train_split)
features = pd.DataFrame(features)
features.head()

train_data = features.loc[0 : train_split - 1]
val_data = features.loc[train_split:]

print("All data: ", features.shape)
print("Train data: ", train_data.shape)
print("Val data: ", val_data.shape)

## Create Train & Validation Datasets


In [ ]:
start = past + future # Defines the start position of the labels, i.e., from the 792nd observation (720 + 72)
end = start + train_split # Defines the end position of the labels

'''
Creates train data and labels
'''
x_train = train_data[[i for i in range(7)]].values
y_train = features.iloc[start:end][[1]] # Labels start from the 792nd observation (720 + 72)

print("\nx_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)


'''
Defines the length of a sequence:
- (720 observations in a sequence) divided by (6 observations per hour)
'''
sequence_length = int(past / step)
print("\nsequence_length: ", sequence_length)

'''
Create a Dataset object for the training data.
For this, we use the 
keras.preprocessing.timeseries_dataset_from_array function.
It takes in a sequence of data-points gathered at equal intervals, 
along with time series parameters such as length of the sequences/windows, 
spacing between two sequence/windows, etc., to produce batches of sub-timeseries inputs and 
targets sampled from the main timeseries.
'''

dataset_train = tf.keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)





'''
Create the validation data
'''
x_end = len(val_data) - past - future
label_start = train_split + past + future


x_val = val_data.iloc[:x_end][[i for i in range(7)]].values
y_val = features.iloc[label_start:][[1]]


'''
Create a Dataset object for the training data.
For this, we use the 
keras.preprocessing.timeseries_dataset_from_array function.
'''
dataset_val = tf.keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)


for batch in dataset_train.take(1):
    inputs, targets = batch

print("\nInput shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

## Create a Transformer Model

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs) 
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
    
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = tf.keras.layers.Dense(dim, activation="relu")(x)
        x = tf.keras.layers.Dropout(mlp_dropout)(x)
        
    # Output layer
    outputs = tf.keras.layers.Dense(1, activation=None)(x)
    
    return tf.keras.Model(inputs, outputs)

In [ ]:
for batch in dataset_train.take(1):
    inputs, targets = batch

print("\nInput shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

input_shape = inputs.numpy().shape[1:]

print("Transformer input_shape: ", input_shape)

model_transformer = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)


model_transformer.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

model_transformer.summary()



## Train the Transformer Model

In [ ]:
path_checkpoint = "Weather_forecasting_half_day_Transformer_checkpoint.h5"
es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

history_transformer = model_transformer.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val,
    callbacks=[es_callback, modelckpt_callback],
)

## Visualize the Train & Validation Loss

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


visualize_loss(history_transformer, "Training and Validation Loss")

## Model Evaluation: using the History Object

In [ ]:
numOfEpochs = len(history_transformer.history['loss'])
print("Epochs: ", numOfEpochs)

train_MAE = history_transformer.history['mean_absolute_error']
val_MAE = history_transformer.history['val_mean_absolute_error']


train_loss = history_transformer.history['loss']
val_loss = history_transformer.history['val_loss']


# Read the last value from the list that represents final epoch statistics
print("\nMAE (train): ", train_MAE[-1])
print("MAE (val): ", val_MAE[-1])

print("\nLoss (train): ", train_loss[-1])
print("Loss (val): ", val_loss[-1])

## Model Evaluation: using the evaluate() Method on the Batched Val Dataset Object

In [ ]:
val_loss_dataset, val_MAE_dataset = model.evaluate(dataset_val, verbose=0)

print("\nLoss (val) from Dataset: ", val_loss_dataset)
print("MAE (val) from Dataset: ", val_MAE_dataset)

## Prediction

We make predictions for 5 sets of values from validation set.

In [ ]:
def show_plot(plot_data, delta, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]
    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    if delta:
        future = delta
    else:
        future = 0

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i:
            plt.plot(future, plot_data[i], marker[i], markersize=10, label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
    plt.legend()
    plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return


# The future step for plotting the prections
future_step = int(future/step)

for x, y in dataset_val.take(5):
    show_plot(
        [x[0][:, 1].numpy(), y[0].numpy(), model_transformer.predict(x)[0]],
        future_step,
        "Single Step Prediction",
    )